In [1]:
#Install requirement package

!pip install requests
!pip install pandas
!pip install io
!pip install collections
!pip install re

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [3]:
#Import package

import requests
import pandas as pd
import csv
import io
from collections import defaultdict
import re

In [42]:
#Set variable

#Destination server
serverurl="https://192.168.1.1"
outputcsv="/api/table.csv?output=csvtable&count=50000"
checkdevice="&content=devices&columns=objid,host,type,device,group,location,active,probe,grpdev,notifiesx,intervalx,accessrights,dependency,probegroupdevice,status,message,priority,upsens,downsens,totalsens,downacksens,partialdownsenswarnsens,pausedsens,unusualsens,undefinedsens,totalsens"
checksensor="&content=sensors&columns=parentid,group,device,sensor,status,message,lastvalue,priority"
linktodevice="/device.htm?id="

#Credential (Make sure you have the login access to PRTG Server)
username=""
passhash=""

#Destination export and column label
exportname="PRTG-Device-Status-Report"
columnlabel=['Group Name', 'Device Name', 'Sensor', 'Status', 'Message', 'Last Value', 'Priority', 'IPv4 Address', 'Location','Link to Device']

In [5]:
#Set link getdevice for destiation server
getdevice=serverurl+outputcsv+checkdevice+'&username='+username+'&passhash='+passhash

#Download data devices
rdevice = requests.get(getdevice,verify = False)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [43]:
#Write csv to memory with StringIO
devices_csv_strio = io.StringIO(rdevice.text)

#Read the csv data devices
devices_csv_reader = csv.reader(devices_csv_strio, delimiter=',')

In [44]:
#Create dictionary for add to data sensor. ParentID as key, IP Address and Location as value 
devices_ip_dict = defaultdict(str)
for device in devices_csv_reader:
    devices_ip_dict[device[0]] = device[2],device[11]

In [22]:
#Set link getsensor for destiation server
getsensor=serverurl+outputcsv+checksensor+'&username='+username+'&passhash='+passhash

#Download data sensors
rsensor = requests.get(getsensor,verify = False)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [45]:
#Write csv to memory with StringIO
sens_strio = io.StringIO(rsensor.text)

#Open the csv data sensor to dataframe
sens_df = pd.read_csv(sens_strio)

#Drop unused duplicate table
sens_df_cln=sens_df.drop(['Parent ID(RAW)', 'Group(RAW)', 'Device(RAW)', 'Sensor(RAW)', 'Status(RAW)', 'Message', 'Last Value(RAW)', 'Priority(RAW)'], axis = 1)

#Export dataframe to csv
sens_csv = sens_df_cln.to_csv(sep=',', index=False, encoding='utf-8')

#Write csv to memory with StringIO
sens_strio = io.StringIO(sens_csv)

#Read the csv data sensors
sens_dict = csv.DictReader(sens_strio)

In [46]:
#Create empty list
output_list = list()

for sens in sens_dict:
    #Create empty list
    device_info = list()

    #Add the devices information to the device info list.
    #device_info.append(sens['Parent ID'])
    device_info.append(sens['Group'])
    device_info.append(sens['Device'])
    device_info.append(sens['Sensor'])
    device_info.append(sens['Status'])
    device_info.append(sens['Message(RAW)'])
    device_info.append(sens['Last Value'])
    device_info.append(sens['Priority'])
    device_info.append(devices_ip_dict[sens['Parent ID']][0])
    device_info.append(devices_ip_dict[sens['Parent ID']][1])
    device_info.append(serverurl+linktodevice+sens['Parent ID'])
    
    #Add the device info list to the output list.
    output_list.append(device_info)

#Create dataframe from output list
output_df = pd.DataFrame(output_list, columns=columnlabel)

#Export data to Excel
output_df.to_excel(exportname + '.xlsx', index=None, header=True)

In [70]:
#Check result
#display(output_df.head())
display(output_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9935 entries, 0 to 9934
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Group Name      9935 non-null   object
 1   Device Name     9935 non-null   object
 2   Sensor          9935 non-null   object
 3   Status          9935 non-null   object
 4   Message         9935 non-null   object
 5   Last Value      9935 non-null   object
 6   Priority        9935 non-null   object
 7   IPv4 Address    9935 non-null   object
 8   Location        9935 non-null   object
 9   Link to Device  9935 non-null   object
dtypes: object(10)
memory usage: 776.3+ KB


None